# Atividade 3 - Case
  
#### Conforme planilha de dados fornecida. 
Deseja-se prospectar empresas que possuam soluções em **tratamento de água** , 
principalmente,  elativas à : **solutions on waste and water, Improve water quality and
water efficiency use, water contamination, water for human consumption, water resources** .

### Declarações Importações de Libs

In [23]:
import re
from pathlib import Path
import pickle
import joblib
import pandas as pd
import spacy
import scipy.sparse
from scipy.sparse import hstack, csr_matrix
from gensim import matutils
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

from sklearn.feature_extraction.text import (TfidfVectorizer,
                                             CountVectorizer)
from sklearn.model_selection import (train_test_split,
                                     GridSearchCV)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import (make_pipeline,
                              Pipeline)
from sklearn.svm import SVC

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
import seaborn as sns

### Declarações de Constantes Globais

In [2]:
ROOT_DIR = Path().absolute()
SOURCE_SUBDIR = 'Data'
DEST_SUBDIR = 'Data'
DATA_FILE_NAME = 'data.parquet'
FULL_DATA_FILE_NAME = ROOT_DIR / SOURCE_SUBDIR / DATA_FILE_NAME

### Criação do Modelo - Exercício 1

In [8]:
# Carregando os dados do Excel
df = pd.read_parquet(FULL_DATA_FILE_NAME)

# Carrega os registros selecionados
df_selected = pd.read_parquet(ROOT_DIR / SOURCE_SUBDIR /'Selected_Data.parquet')

# Carrega os registros rotulados, deve conter os indices originais
df_final = pd.read_parquet(ROOT_DIR / SOURCE_SUBDIR /'Labeled_Data.parquet')

tfidf_matrix = joblib.load(ROOT_DIR / SOURCE_SUBDIR / 'tfidf_matrix.pkl')
# Carrega o vetorizador alinhado com os dados selecionados
tfidf_matrix_restrita = joblib.load(ROOT_DIR / SOURCE_SUBDIR / 'tfidf_matrix_restrita.pkl')

In [9]:
# Selecionando características para modelagem
X = df_final[['original_water_term_count', 'altered_water_term_count', 'extended_water_term_count', 'topic']]
# Convertendo o DataFrame de características em matriz esparsa
X_sparse = csr_matrix(X.values)

# Concatenando a matriz esparsa de características com a matriz TF-IDF restrita
X_combined = hstack([X_sparse, tfidf_matrix_restrita])
# Dividindo o conjunto de dados combinado em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_combined, df_final['label'], test_size=0.2, random_state=42)

In [10]:
# Criando um dataframe com todos os dados
df_final = df.merge(df_selected[['label']], left_index=True, right_index=True, how='inner')
# Selecionando características para modelagem
X = df_final[['original_water_term_count', 'altered_water_term_count', 'extended_water_term_count', 'topic']]
# Convertendo o DataFrame de características em matriz esparsa
X_sparse = csr_matrix(X.values)
# Restringir tfidf_matrix aos registros rotulados
#tfidf_matrix_restrita = tfidf_matrix[df_final.index]

# Concatenando a matriz esparsa de características com a matriz TF-IDF restrita
X_combined = hstack([X_sparse, tfidf_matrix_restrita])
# Dividindo o conjunto de dados combinado em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_combined, df_final['label'], test_size=0.2, random_state=42)

#### Regressão Logística (Melhor caso)

Este será o modelo escolhido devido a ter a melhor relação Acurácia e Recall. Está longe de 
ser um modelo bom, para isto seria necessário um trabalho de reavaliação das features extraídas,
do preprocessamento de texto e outros pontos.

In [12]:
# Definindo o modelo
log_reg = LogisticRegression(max_iter=1000, multi_class='multinomial')

# Definindo o espaço de hiperparâmetros para exploração
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'saga']
}

# Configurando o GridSearchCV
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy', verbose=1)

# Realizando o GridSearchCV com os dados de treinamento
grid_search.fit(X_train, y_train)

# Verificando os melhores parâmetros encontrados
print("Melhores parâmetros:", grid_search.best_params_)

# Avaliando o desempenho do melhor modelo encontrado no conjunto de teste
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_

Melhores parâmetros: {'C': 0.1, 'solver': 'saga'}
                 precision    recall  f1-score   support

Menos relevante       0.00      0.00      0.00         1
  Não relevante       1.00      1.00      1.00         5
      Relevante       0.67      1.00      0.80         2

       accuracy                           0.88         8
      macro avg       0.56      0.67      0.60         8
   weighted avg       0.79      0.88      0.82         8



/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.p

In [13]:
# Salvando o melhor modelo encontrado pelo GridSearchCV
joblib.dump(best_model, ROOT_DIR / SOURCE_SUBDIR / 'best_logistic_regression_model.pkl')

# Salvando o vetorizador
joblib.dump(tfidf_matrix_restrita, ROOT_DIR / SOURCE_SUBDIR / 'tfidf_matrix_restrita.pkl')

['/media/pabloernesto/Seagate Expansion Drive/Work/Projects/Desafios/FirstDecision/Prospect-Canadian-Water-Companies/Data/tfidf_matrix_restrita.pkl']

A identificação de não relevantes é ótima (na verdade perfeita demais), a de relevantes não tão boa, mas a de menos relevantes está nula... 
Irei testar outro algoritmo.


#### Random forest

In [15]:
# Inicializar o modelo de Florestas Aleatórias
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo com o conjunto de treinamento
random_forest.fit(X_train, y_train)

# Prever os rótulos para o conjunto de teste
y_pred_rf = random_forest.predict(X_test)

# Avaliar o desempenho do modelo
print(classification_report(y_test, y_pred_rf))

                 precision    recall  f1-score   support

Menos relevante       0.00      0.00      0.00         1
  Não relevante       0.71      1.00      0.83         5
      Relevante       1.00      0.50      0.67         2

       accuracy                           0.75         8
      macro avg       0.57      0.50      0.50         8
   weighted avg       0.70      0.75      0.69         8



/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

A identificação de não relevantes é ótima, a de relevantes não tão boa, mas a de menos relevantes está nula (novamente)... 
Vou tentar fazer um balanceamento de calsses usando SMOTE

#### SMOTE (Para balanceamento de classes)

In [17]:
# Supondo que a classe minoritária tenha 5 amostras, ajuste n_neighbors para 4 ou menos
smote = SMOTE(random_state=42, k_neighbors=4)

# Aplicando o SMOTE ajustado ao conjunto de treinamento
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Verificando a nova distribuição das classes
print(pd.Series(y_train_smote).value_counts())

label
Relevante          16
Não relevante      16
Menos relevante    16
Name: count, dtype: int64


#### Reavaliando Logistic Regression (com SMOTE)

In [18]:
# Inicializando o modelo de Regressão Logística com os melhores parâmetros encontrados anteriormente
log_reg_balanced = LogisticRegression(C=0.1, solver='saga', max_iter=1000, multi_class='multinomial')

# Treinando o modelo com os dados balanceados
log_reg_balanced.fit(X_train_smote, y_train_smote)

# Previsões no conjunto de teste
y_pred_log_reg = log_reg_balanced.predict(X_test)

# Avaliando o desempenho
print("Regressão Logística com Dados Balanceados:")
print(classification_report(y_test, y_pred_log_reg))

Regressão Logística com Dados Balanceados:
                 precision    recall  f1-score   support

Menos relevante       0.00      0.00      0.00         1
  Não relevante       1.00      0.80      0.89         5
      Relevante       0.50      1.00      0.67         2

       accuracy                           0.75         8
      macro avg       0.50      0.60      0.52         8
   weighted avg       0.75      0.75      0.72         8



/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Não houve mudança com os dados balanceados... Podeindicar overfiting dos dados criados pelo SMOTE. 

#### Redes Neurais (com SMOTE)

In [20]:
# Criando um pipeline com StandardScaler configurado corretamente para matrizes esparsas e MLPClassifier
mlp_pipeline = make_pipeline(StandardScaler(with_mean=False), MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', max_iter=1000, random_state=42))

# Treinando o pipeline nos dados balanceados pelo SMOTE
mlp_pipeline.fit(X_train_smote, y_train_smote)

# Fazendo previsões no conjunto de teste
y_pred_mlp = mlp_pipeline.predict(X_test)

# Avaliando o desempenho do MLPClassifier
print("Desempenho do MLPClassifier após correção:")
print(classification_report(y_test, y_pred_mlp))


Desempenho do MLPClassifier após correção:
                 precision    recall  f1-score   support

Menos relevante       0.00      0.00      0.00         1
  Não relevante       0.67      0.80      0.73         5
      Relevante       0.50      0.50      0.50         2

       accuracy                           0.62         8
      macro avg       0.39      0.43      0.41         8
   weighted avg       0.54      0.62      0.58         8



/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

#### SVM (com SMOTE)

In [22]:
# Definindo o pipeline com padronização e SVC
svm_pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),  # with_mean=False é importante para dados esparsos
    ('svc', SVC(kernel='linear', C=1))
])

# Treinando o pipeline nos dados balanceados pelo SMOTE
svm_pipeline.fit(X_train_smote, y_train_smote)

# Fazendo previsões no conjunto de teste
y_pred_svm = svm_pipeline.predict(X_test)

# Avaliando o desempenho do SVM
print("Desempenho do SVM:")
print(classification_report(y_test, y_pred_svm))


Desempenho do SVM:
                 precision    recall  f1-score   support

Menos relevante       0.00      0.00      0.00         1
  Não relevante       0.67      0.80      0.73         5
      Relevante       0.50      0.50      0.50         2

       accuracy                           0.62         8
      macro avg       0.39      0.43      0.41         8
   weighted avg       0.54      0.62      0.58         8



/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pabloernesto/anaconda3/envs/firstenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Resultado parecido ao de Redes Neurais...

### Conclusão Final

O modelo escolhiodo é a Regressão logistica devido aos seguintes fatores:

Maior acurácia geral de 0.88 (88%)

O modelo teve uma precisão e recall de 1.00 (100%) para a classe "Não relevante", identificando corretamente todos os registros dessa classe. Isto pde ser devido aos dados de TF-IDF e sua associação com o tópico do documento, já que o tópico 3 não continha nenhum caso de emprea reslacionada ao objetivo, assim apontava a aqueles tópicos com menor probabilidade de conter uma empresa relevante.

Com relação à classe "Relevante", com a precisão foi de 0.67 (67%) e o recall alcançando 1.00. Isso indica que, embora o modelo ainda faça algumas previsões incorretas para essa classe, ele foi capaz de identificar corretamente todos os registros "Relevantes" no conjunto de teste. O caso da precisão menor, ao contrario do "não relevante", possivelmente se deve a existerem registros de empresas relevantes em 3 dos 4 tópicos, e ao fato de que o número de registros relevantes é desproporcionalmente baixo se comparado ao número de registros "não relevantes".

A classe com identificação mais problemática é a de "Menos relevante", com o modelo não conseguindo identificar corretamente nenhum registro dessa classe. Isso pode ser devido à falta de exemplos suficientes dessa classe no conjunto de treinamento ou a características menos distintivas que dificultam a diferenciação pelo modelo.

Se comparado ao teste inicial, sem fine-tunig, as médias macro avg e weighted avg, assim como precisão e recall, mostram uma melhoria geral no desempenho do modelo em relação a todas as classes, refletindo os ganhos nas classes "Não relevante" e "Relevante". O primeiro teste, não incluído aqui, foi uma execução direta da Regressão Logística, sem fine-tuning.

O quê poderia ser feito para melhorar o modelo: A "rotulação" de mais amostras, um melhor estudo qualitativo dos textos buscando outros termos associados, testes utilizando outras abordagens como similaridade entre os textos, para isto eu já tenho o TF-IDF, bastaria calcular a distância (possivelmente usando cosseno), desta forma poderia até criar escores alinhados a faixas de distâncias, a vantagem é que se vê válida especialmente pelas poucas amostras que se têm, mas exige maior tempo de análise. Na mesma linha poderia se aplicar clusterização, O k-means já faria esse agrupamento pelas distâncias.


|Regressão Logística                                         |
|------------------------------------------------------------|
|Melhores parâmetros: {'C': 0.1, 'solver': 'saga'}           |

|                 |  precision |  recall | f1-score | support|
|:-:|:-:|:-:|:-|:-|
| Menos relevante |     0.00   |   0.00  |    0.00  |      1 |
|  Não relevante  |     1.00   |   1.00  |    1.00  |      5 |
|      Relevante  |     0.67   |   1.00  |    0.80  |      2 |
|------------------------------------------------------------|
|       accuracy  |            |          |   0.88  |      8 |
|      macro avg  |     0.56   |   0.67   |   0.60  |      8 |
|   weighted avg  |     0.79   |   0.88   |   0.82  |      8 |
